In [1]:
# Import all necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import mplfinance as mpf
import plotly.graph_objects as go
import pandas as pd
import cv2
import matplotlib.image as mpimg
import os
from PIL import Image

In [2]:
# Run locally
folder_link = ".."

In [ ]:
train_data = []
train_output_arr = []
for coin in ["ada", "btc", "doge", "dot", "eth", "luna", "matic", "shib", "sol", "xrp"]:
    train_data.append(np.memmap(f"{folder_link}/image_data/{coin}usdt/training_data.dat", dtype = np.uint8, mode = 'r').reshape((-1, 54, 72)))
    train_data.append(np.memmap(f"{folder_link}/image_data/{coin}usdt/validation_data.dat", dtype = np.uint8, mode = 'r').reshape((-1, 54, 72)))
    train_output_arr.append(pd.read_csv(f"{folder_link}/image_data/{coin}usdt/training_output.csv"))
    train_output_arr.append(pd.read_csv(f"{folder_link}/image_data/{coin}usdt/validation_output.csv"))

train_data = np.concatenate(train_data)
print(train_data.shape)

test_data = []
test_output_arr = []
for coin in ["ada", "btc", "doge", "dot", "eth", "luna", "matic", "shib", "sol", "xrp"] + ["bonk", "floki", "pepe", "wif"]:
    test_data.append(np.memmap(f"{folder_link}/image_data/{coin}usdt/testing_data.dat", dtype = np.uint8, mode = 'r').reshape((-1, 54, 72)))
    test_output_arr.append(pd.read_csv(f"{folder_link}/image_data/{coin}usdt/testing_output.csv"))

test_data = np.concatenate(test_data)
print(test_data.shape)

FileNotFoundError: [Errno 2] No such file or directory: '../image_data/adausdt/training_data.dat'

In [ ]:
def normalize(df):
    df = df.drop(["Volume", "Close time", "Quote asset volume", 'Number of trades', "Taker buy base asset volume", "Taker buy quote asset volume"], axis = 1)
    df["Max24"] = df["High"].shift(1).rolling(window = 24).max()
    df["Max24"] = df["Max24"].fillna(df.loc[23, "Max24"])
    df["Min24"] = df["Low"].shift(1).rolling(window = 24).min()
    df["Min24"] = df["Min24"].fillna(df.loc[23, "Min24"])
    df["Scale"] = np.log(df["Max24"]) - np.log(df["Min24"])
    df["NormalizedLR"] = df["LogReturns"] / df["Scale"]

for i in range(len(train_output_arr)):
    normalize(train_output_arr[i])
    normalize(test_output_arr[i])

train_output = pd.concat(train_output_arr)
test_output = pd.concat(test_output_arr)

In [5]:
train_output_arr[0]

IndexError: list index out of range

In [ ]:
# Save train_data
np.savez_compressed("train_data", train_data, allow_pickle = False)
train_output.to_csv("train_output.csv", index = False)

# Save test_data
np.savez_compressed("test_data", test_data, allow_pickle = False)
test_output.to_csv("test_output.csv", index = False)